## Eur-Lex : 3 principales options disponibles:


1 - Service Web EUR-Lex : d'effectuer des recherches et de récupérer des métadonnées sur les documents juridiques. Ce service offre des options de requête similaires à celles de la recherche experte sur le site web et délivre les données au format XML.

2 - API RESTful de Cellar : permet de récupérer des ensembles spécifiques de métadonnées (notices) et de télécharger le contenu des documents dans divers formats (PDF, HTML, etc.). Est accessible sans inscription préalable


3 - Point d'accès SPARQL : Pour des requêtes plus complexes, EUR-Lex met à disposition un point d'accès SPARQL permettant d'interroger directement les métadonnées disponibles dans le référentiel Cellar. Cela offre une flexibilité accrue pour extraire des informations précises sur les documents juridiques

### 1.1 - Méthode 1 - SPARQL

Deux fonctions: 
- celextourl_data
- celextourl_getlink_full
- celextourl_getlink

In [85]:
from SPARQLWrapper import SPARQLWrapper, JSON

def celextourl_data(endpoint_url, celex_id):

    # Préparer la requête SPARQL
    query = f"""
    PREFIX cdm: <http://publications.europa.eu/ontology/cdm#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    SELECT ?property ?value
    WHERE {{
      ?acte cdm:resource_legal_id_celex "{celex_id}"^^xsd:string ;
            ?property ?value .
    }}
    """

    # Initialiser le wrapper SPARQL
    sparql = SPARQLWrapper(endpoint_url) #inject url endpoint
    sparql.setQuery(query)               # inject query
    sparql.setReturnFormat(JSON)          # set in json format
    results = sparql.query().convert()   # run
    return results

In [86]:
json_res= celextourl_data('https://publications.europa.eu/webapi/rdf/sparql', "32019L2177")

In [81]:
def celextourl_getlink_full(sparql_json):
    bindings = sparql_json["results"]["bindings"]
    return {
        "date_creation": next(b["value"]["value"] for b in bindings if b["property"]["value"] == "http://publications.europa.eu/ontology/cdm#date_creation_legacy"),
        "lien_adopted": next(b["value"]["value"] for b in bindings if b["property"]["value"] == "http://www.w3.org/2002/07/owl#sameAs" ),
        "date_entry_into_force": next(b["value"]["value"] for b in bindings if b["property"]["value"] == "http://publications.europa.eu/ontology/cdm#resource_legal_date_entry-into-force"),
        "date_signature": next(b["value"]["value"] for b in bindings if b["property"]["value"] == "http://publications.europa.eu/ontology/cdm#resource_legal_date_signature")
    }


In [82]:
celextourl_getlink_full(json_res)

{'date_creation': '2019-12-27',
 'lien_adopted': 'http://publications.europa.eu/resource/celex/32019L2177',
 'date_entry_into_force': '2019-12-30',
 'date_signature': '2019-12-18'}

In [83]:
def celextourl_getlink(sparql_json):
    bindings = sparql_json["results"]["bindings"]
    return next(b["value"]["value"] for b in bindings
                if b["property"]["value"] == "http://www.w3.org/2002/07/owl#sameAs" )

In [84]:
celextourl_getlink(json_res)

'http://publications.europa.eu/resource/celex/32019L2177'